In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import rushd as rd
import scipy as sp
import seaborn as sns

from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'qPCR'/'emma'/'command'
output_path = rd.rootdir/'output'/'qPCR'/'kasey'
cache_path = output_path/'data.gzip'

plates = pd.DataFrame({
    'data_path': [base_path/'2024.11.13_command'/'2024.11.13_ELP_command_qPCR_Cp.txt', base_path/'2024.11.22_command'/'2024.11.22_ELP_command_qPCR_Cp.txt'],
    'yaml_path': [base_path/'2024.11.13_command'/'wells.yaml', base_path/'2024.11.22_command'/'wells.yaml'],
})

for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)

In [ ]:
data = pd.DataFrame()
if cache_path.exists(): data = pd.read_parquet(cache_path)
else: 
    data = base.load_data_qpcr(plates)
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

ts_label = {'na': 'base', 'NT': 'OL', 'T': 'CL', 'none': '–'}

In [ ]:
def get_housekeeping(df, gene='GAPDH', stat=np.mean):
    df['housekeeping_Cp'] = df.loc[df['primers']=='GAPDH', 'Cp'].agg(stat)
    return df

data = data.groupby(['dox','construct','biorep'])[data.columns].apply(get_housekeeping).reset_index(drop=True)
data['delta_Cp'] = data['Cp'] - data['housekeeping_Cp']
data['expression'] = data['Cp'].apply(lambda x: 2**(-x))
data['norm_expression'] = data['delta_Cp'].apply(lambda x: 2**(-x))

In [ ]:
def get_control(df, construct='UI', stat=np.median):
    df[construct+'_Cp'] = df.loc[(df['construct']==construct), 'delta_Cp'].agg(stat)
    return df

data = data.groupby(['dox','primers','biorep'])[data.columns].apply(lambda x: get_control(x, 'RC281')).reset_index(drop=True)
data = data.groupby(['dox','primers','biorep'])[data.columns].apply(lambda x: get_control(x, 'RC284')).reset_index(drop=True)
data['delta_delta_Cp'] = data['delta_Cp'] - data['RC281_Cp']
data.loc[data['construct'].isin(['RC284','RC285','RC286']), 'delta_delta_Cp'] = data.loc[data['construct'].isin(['RC284','RC285','RC286']), 'delta_Cp'] - data.loc[data['construct'].isin(['RC284','RC285','RC286']), 'RC284_Cp']
data['relative_expression'] = data['delta_delta_Cp'].apply(lambda x: 2**(-x))

In [ ]:
# Plot controls
plot_df = data[data['construct'].isin(['no-RT','no-cDNA','UI'])]
g = sns.catplot(data=plot_df, x='primers', y='expression', hue='construct',
                col='biorep', kind='strip', dodge=True)
for _,ax in g.axes_dict.items():
    ax.axhline(2**(-35), c='black', ls=':', zorder=0) # max cycle number
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
g.figure.savefig(rd.outfile(output_path/f'expression_controls.png'))

In [ ]:
primers_list = ['FXN', 'mRuby2-1', 'mRuby2-2']
for biorep in data['biorep'].unique():
    fig, axes = plt.subplots(1, len(primers_list), figsize=(10,4), gridspec_kw=dict(wspace=0.5))

    for ax, primers in zip(axes, primers_list):
        plot_df = data[(data['primers']==primers) & ~(data['construct'].isin(['no-RT','no-cDNA','UI'])) & (data['biorep']==biorep)]
        sns.stripplot(data=plot_df, x='ts_kind', y='norm_expression', hue='dox', 
                    ax=ax, legend=False)
        ax.set(xlabel='', ylabel='', title=primers)
        ax.set_xticklabels([ts_label[l.get_text()] for l in ax.get_xticklabels()], rotation=45, ha='right')
        baseline = data.loc[(data['primers']==primers) & (data['construct']=='UI') & (data['biorep']==biorep), 'norm_expression'].median()
        ax.axhline(baseline, c='black', ls=':', zorder=0)
        sns.despine(ax=ax)
    fig.savefig(rd.outfile(output_path/f'norm-expression_FXN_biorep{biorep}.png'))

In [ ]:
primers_list = ['FMRP-1', 'FMRP-2', 'EGFP-1', 'EGFP-2']
for biorep in data['biorep'].unique():
    fig, axes = plt.subplots(1, len(primers_list), figsize=(12,4), gridspec_kw=dict(wspace=0.5))

    for ax, primers in zip(axes, primers_list):
        plot_df = data[(data['primers']==primers) & ~(data['construct'].isin(['no-RT','no-cDNA','UI'])) & (data['biorep']==biorep)]
        sns.stripplot(data=plot_df, x='ts_kind', y='norm_expression', hue='dox', 
                    ax=ax, legend=False)
        ax.set(xlabel='', ylabel='', title=primers)
        ax.set_xticklabels([ts_label[l.get_text()] for l in ax.get_xticklabels()], rotation=45, ha='right')
        baseline = data.loc[(data['primers']==primers) & (data['construct']=='UI') & (data['biorep']==biorep), 'norm_expression'].median()
        ax.axhline(baseline, c='black', ls=':', zorder=0)
        sns.despine(ax=ax)
    fig.savefig(rd.outfile(output_path/f'norm-expression_FMRP_biorep{biorep}.png'))

In [ ]:
primers_list = ['FXN', 'mRuby2-1', 'mRuby2-2']
for biorep in data['biorep'].unique():
    fig, axes = plt.subplots(1, len(primers_list), figsize=(10,4), gridspec_kw=dict(wspace=0.5))

    for ax, primers in zip(axes, primers_list):
        plot_df = data[(data['primers']==primers) & ~(data['construct'].isin(['no-RT','no-cDNA','UI'])) & (data['biorep']==biorep)]
        sns.stripplot(data=plot_df, x='ts_kind', y='relative_expression', hue='dox', 
                    ax=ax, legend=False)
        ax.set(xlabel='', ylabel='', title=primers)
        ax.set_xticklabels([ts_label[l.get_text()] for l in ax.get_xticklabels()], rotation=45, ha='right')
        baseline = data.loc[(data['primers']==primers) & (data['construct']=='UI') & (data['biorep']==biorep), 'relative_expression'].median()
        ax.axhline(baseline, c='black', ls=':', zorder=0)
        sns.despine(ax=ax)
    fig.savefig(rd.outfile(output_path/f'relative-expression_FXN_biorep{biorep}.png'))

In [ ]:
primers_list = ['FMRP-1', 'FMRP-2', 'EGFP-1', 'EGFP-2']
for biorep in data['biorep'].unique():
    fig, axes = plt.subplots(1, len(primers_list), figsize=(12,4), gridspec_kw=dict(wspace=0.5))

    for ax, primers in zip(axes, primers_list):
        plot_df = data[(data['primers']==primers) & ~(data['construct'].isin(['no-RT','no-cDNA','UI'])) & (data['biorep']==biorep)]
        sns.stripplot(data=plot_df, x='ts_kind', y='relative_expression', hue='dox', 
                    ax=ax, legend=False)
        ax.set(xlabel='', ylabel='', title=primers)
        ax.set_xticklabels([ts_label[l.get_text()] for l in ax.get_xticklabels()], rotation=45, ha='right')
        baseline = data.loc[(data['primers']==primers) & (data['construct']=='UI') & (data['biorep']==biorep), 'relative_expression'].median()
        ax.axhline(baseline, c='black', ls=':', zorder=0)
        sns.despine(ax=ax)
    fig.savefig(rd.outfile(output_path/f'relative-expression_FMRP_biorep{biorep}.png'))

In [ ]:
# Calculate stats for bioreps
stats = data.groupby(['construct','dox','primers','biorep'])[['Cp','housekeeping_Cp','delta_Cp','expression','norm_expression','relative_expression']].median().reset_index()
stats = stats.merge(metadata, how='left', on='construct')
display(stats)

In [ ]:
display(stats[stats['construct']=='RC281'])

In [ ]:
primers_list = ['FXN', 'mRuby2-1', 'mRuby2-2']

fig, axes = plt.subplots(1, len(primers_list), figsize=(12,4), gridspec_kw=dict(wspace=0.5))

for ax, primers in zip(axes, primers_list):
    plot_df = stats[(stats['primers']==primers) & ~(stats['construct'].isin(['no-RT','no-cDNA','UI']))]
    sns.stripplot(data=plot_df, x='ts_kind', y='norm_expression', hue='dox', 
                ax=ax, legend=False)
    ax.set(xlabel='', ylabel='', title=primers)
    ax.set_xticklabels([ts_label[l.get_text()] for l in ax.get_xticklabels()], rotation=45, ha='right')
    baseline = stats.loc[(stats['primers']==primers) & (stats['construct']=='UI'), 'norm_expression'].mean()
    ax.axhline(baseline, c='black', ls=':', zorder=0)
    sns.despine(ax=ax)
fig.savefig(rd.outfile(output_path/f'norm-expression_FXN.png'))

In [ ]:
primers_list = ['FMRP-1', 'FMRP-2', 'EGFP-1', 'EGFP-2']

fig, axes = plt.subplots(1, len(primers_list), figsize=(12,4), gridspec_kw=dict(wspace=0.5))

for ax, primers in zip(axes, primers_list):
    plot_df = stats[(stats['primers']==primers) & ~(stats['construct'].isin(['no-RT','no-cDNA','UI']))]
    sns.stripplot(data=plot_df, x='ts_kind', y='norm_expression', hue='dox', 
                ax=ax, legend=False)
    ax.set(xlabel='', ylabel='', title=primers)
    ax.set_xticklabels([ts_label[l.get_text()] for l in ax.get_xticklabels()], rotation=45, ha='right')
    baseline = stats.loc[(stats['primers']==primers) & (stats['construct']=='UI'), 'norm_expression'].mean()
    ax.axhline(baseline, c='black', ls=':', zorder=0)
    sns.despine(ax=ax)
fig.savefig(rd.outfile(output_path/f'norm-expression_FMRP.png'))

In [ ]:
primers_list = ['FXN', 'mRuby2-1', 'mRuby2-2']

fig, axes = plt.subplots(1, len(primers_list), figsize=(12,4), gridspec_kw=dict(wspace=0.5))

for ax, primers in zip(axes, primers_list):
    plot_df = stats[(stats['primers']==primers) & ~(stats['construct'].isin(['no-RT','no-cDNA','UI']))]
    sns.stripplot(data=plot_df, x='ts_kind', y='relative_expression', hue='dox', 
                ax=ax, legend=False)
    ax.set(xlabel='', ylabel='', title=primers)
    ax.set_xticklabels([ts_label[l.get_text()] for l in ax.get_xticklabels()], rotation=45, ha='right')
    baseline = stats.loc[(stats['primers']==primers) & (stats['construct']=='UI'), 'relative_expression'].mean()
    ax.axhline(baseline, c='black', ls=':', zorder=0)
    sns.despine(ax=ax)
fig.savefig(rd.outfile(output_path/f'relative-expression_FXN.png'))

In [ ]:
primers_list = ['FMRP-1', 'FMRP-2', 'EGFP-1', 'EGFP-2']

fig, axes = plt.subplots(1, len(primers_list), figsize=(12,4), gridspec_kw=dict(wspace=0.5))

for ax, primers in zip(axes, primers_list):
    plot_df = stats[(stats['primers']==primers) & ~(stats['construct'].isin(['no-RT','no-cDNA','UI']))]
    sns.stripplot(data=plot_df, x='ts_kind', y='relative_expression', hue='dox', 
                ax=ax, legend=False)
    ax.set(xlabel='', ylabel='', title=primers)
    ax.set_xticklabels([ts_label[l.get_text()] for l in ax.get_xticklabels()], rotation=45, ha='right')
    baseline = stats.loc[(stats['primers']==primers) & (stats['construct']=='UI'), 'relative_expression'].mean()
    ax.axhline(baseline, c='black', ls=':', zorder=0)
    sns.despine(ax=ax)
fig.savefig(rd.outfile(output_path/f'relative-expression_FMRP.png'))